<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Сравнение-качества-моделей-до-и-после-преобразования" data-toc-modified-id="Сравнение-качества-моделей-до-и-после-преобразования-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Сравнение качества моделей до и после преобразования</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем библиотеки с которыми будем работать

In [ ]:
import pandas as pd
import numpy as np

from scipy import stats as st
from scipy.stats import multivariate_normal
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

Загрузим датасет

In [ ]:
insurance = pd.read_csv('/datasets/insurance.csv', sep=',')

Выведем общую информацию о датасете и первые 5 строк

In [ ]:
display(insurance.info())
display(insurance.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Выведем на экран основные значения по датасету

In [ ]:
display(insurance.describe().T)

,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


Определим, есть ли явные дубликаты

In [ ]:
print('Количество дубликатов', insurance.duplicated().sum())

Количество дубликатов 153


### Вывод:
 - Датасет содержит 5000 строк, 5 столбцов
 - Из описания к проекту нам известно, что Признаки:
   - пол
   - возраст
   - зарплата застрахованного
   - количество членов его семьи
 - Целевой признак: количество страховых выплат клиенту за последние 5 лет
 - В столбце **Возраст** и **Зарплата** тип данных `float64`, необходимо будет заменить на `int`

### Предобработка данных

Избавимся от явных дубликатов

In [ ]:
insurance.drop_duplicates(inplace = True)
print('Количество дубликатов', insurance.duplicated().sum())

Количество дубликатов 0


Заменим тип данных в столбце **Возраст** и **Зарплата**

In [ ]:
insurance['Возраст']=insurance['Возраст'].astype('int')
insurance['Зарплата']=insurance['Зарплата'].astype('int')
print(insurance.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB
None


#### Вывод:
 - Избавились от дубликатов, изменили тип данных в столбцах **Возраст** и **Зарплата**

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.*

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:** Для обоснования нашего утверждения в формулах предсказания и обучения заменим X на XP и введем новые обозначения $a′$ и $w′$:  
$$a′ = XPw′$$
$$w' = ((XP)^T XP)^{-1}(XP)^T y$$

Подставим значение формулы $w′$ в $a′$:
$$
a' = X P ((XP)^T XP)^{-1}(XP)^T y $$

Преобразуем получившуюся формулу, пользуясь свойствами обратной $^*$, транспонированной$^*$$^*$ и еденичной$^*$$^*$$^*$ матриц:

$$a' = X P (P^TX^T XP)^{-1}P^T X^T y = X P (X^TXP)^{-1} (P^T)^{-1} P^T X^T y = X P P^{-1}(X^TX)^{-1} (P^T)^{-1} P^T X^T y = X E (X^TX)^{-1} E  X^T y = X (X^TX)^{-1} X^T y$$

$^*$  $(AB)^{-1} = B^{-1} A^{-1}$ (для двух квадратных обратимых матриц A и B)

$^*$$^*$  $(AB)^T = B^T A^T$

$^*$$^*$$^*$ $(АА)^{-1}=E$; $AE = EA = A$

(4) После всех преобразований видим, что: $$a' = X (X^TX)^{-1} X^T y = Xw = a$$

### Вывод:
 - При умножении признаков на обратимую матрицу качество линейной регрессии не изменится

## Алгоритм преобразования

**Алгоритм**

В общем случае, алгоритм шифрования может быть выражен в следующем виде:
$$C=KP$$  
Где P и C - открытая и зашифрованная матрица, K - квадратная матрица, представляющая ключ шифрования.

**Обоснование**

Умножая открытую матрицу на ключ мы сдвигаем пространство признаков, не меняя общей структуры.  
Следовательно качество модели никак не изменится.   
Расшифровка: $$ P = K^{-1}C = K^{-1}KP = P$$
https://ru.wikipedia.org/wiki/Шифр_Хилла

**Алгоритм**
1. Можно сгенерировать обратимую матрицу, соответственно квадратную, размерностью соответствующую количеству столбцов признаков, а именно 4х4.
2. Умножить матрицу исходных признаков на сгенерированную матрицу.
3. Посчитать качество модели на полученной матрице и сравнить с качеством модели на матрице исходных данных.  
  
**Обоснование**  
В предыдущем пункте мы выяснили, что предсказания на измененной и изначальной матрице не должны отличаться, поэтому результаты метрики R2 должны оказаться равны. Тем самым мы докажем, что правильно защитили наши данные.

**Проверка алгоритма**

Создадим исходные признаки

In [ ]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

Сгенерируем матрицу размером 4х4 методом random.normal (), т.к. при таком подходе вероятность получить необратимую матрицу близка к нулю

In [ ]:
matrix_generated = np.random.normal(size=(4, 4))
display (matrix_generated)

array([[ 0.84881437,  0.5422245 ,  0.09444359,  1.68604108],
       [ 0.88785708, -0.01145502,  0.93903271, -0.94014909],
       [-0.56889633, -0.59937295, -0.9358034 , -0.70642308],
       [-2.05418717,  0.67398588,  0.39178488, -0.34209593]])

Сделаем из нашей сгенерированной матрицы обратную методом linalg.inv(), т.к. в процессе обращения мы одновременно проверим матрицу на обратимость, если появится ошибка, то изначально наша сгенерированная матрица не являлась обратимой.

In [ ]:
matrix_invert = np.linalg.inv(matrix_generated)
display (matrix_invert)

array([[ 1.49575302,  1.36087858,  1.62887378,  0.2683407 ],
       [ 5.47202147,  4.39263975,  6.00526053,  2.49655735],
       [-3.09584059, -2.1480054 , -3.99724879, -1.10062785],
       [-1.7462792 , -1.97745173, -2.52739769, -0.87632429]])

Создадим матрицу из наших изначальных признаков, перемножим ее на сгенерированную и обращенную матрицу и составим из полученной новый набор измененных признаков

In [ ]:
matrix = features.values
matrix_new = matrix@matrix_invert
features_incode = pd.DataFrame (matrix_new, columns = features.columns)
features_incode

,Пол,Возраст,Зарплата,Члены семьи
0,-153329.591149,-106361.586258,-198018.222809,-54489.390506
1,-117391.975895,-81424.121280,-151621.739420,-41709.892996
2,-64853.963869,-44980.726879,-83768.072027,-23040.784693
3,-128985.132911,-89483.534711,-166564.218853,-45845.506300
4,-80646.727171,-55938.586188,-104158.417241,-28656.214946
...,...,...,...,...
4842,-110371.785192,-76564.753824,-142538.689291,-39224.263297
4843,-162037.744718,-112408.110739,-209254.185118,-57588.892728
4844,-104843.048293,-72733.485219,-135391.683554,-37263.105626
4845,-101117.346063,-70147.710031,-130583.873009,-35937.967074


Создадим класс LinearRegression для последующего обучения наших моделей

In [ ]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Обучим модель на изначальных признаках и выведем на экран метрику качества R2

In [ ]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010046633359


Обучим модель на закодированных признаках и выведем на экран метрику качества R2

In [ ]:
model = LinearRegression()
model.fit(features_incode, target)
predictions = model.predict(features_incode)
print(r2_score(target, predictions))

0.4302010046370789


**Вывод**: Есть некоторая погрешность при обратном преобразовании.

## Проверка алгоритма

Создадим переменные для признаков и целевого признака

In [ ]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

Создадим квадратную матрицу P

In [ ]:
P = np.random.normal(size=(4, 4))
display(pd.DataFrame(P))

,0,1,2,3
0,1.638437,0.912187,-1.009540,-1.100359
1,0.474548,0.220428,-0.667805,-0.324007
2,0.856870,-0.791734,-0.410454,-1.850848
3,1.764771,1.961887,2.049913,0.217198


Проверим, что при умножении данной матрицы на обратную ей получится единичная матрица E

In [ ]:
display(round(pd.DataFrame(P @ np.linalg.inv(P))).astype('int'))

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


Закодируем признаки features, умножив на матрицу P

In [ ]:
features_encoded = features @ P

Посмотрим на исходные признаки

In [ ]:
print('Исходные признаки:')
display(features.head())

Исходные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Посмотрим на закодированные признаки

In [ ]:
print('Закодированные признаки:')
display(features_encoded.head())

Закодированные признаки:


,0,1,2,3
0,42523.596833,-39258.108052,-20384.864240,-91816.244513
1,32584.642614,-30073.800589,-15625.925875,-70346.923589
2,18008.025613,-16620.027204,-8638.902975,-38877.211092
3,35744.961566,-33006.766203,-17125.861096,-77186.745022
4,22379.224973,-20657.180212,-10732.560748,-48317.313916


### Вывод:
 - исходные признаки удалось закодировать

Попробуем восстановить исходные признаки умножив их на обран

In [ ]:
features_recovered = features_encoded @ np.linalg.inv(P)
print('Восстановленные признаки')
display(round(features_recovered).astype('int').head())

Восстановленные признаки


,0,1,2,3
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Данные удалось декодировать - алгоритм работает

### Сравнение качества моделей до и после преобразования

Создадим переменные для признаков и целевого признака

In [ ]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

Обучим модель линейной регрессии

In [ ]:
model = LinearRegression()
model.fit(features, target)

LinearRegression()

Посчитаем метрику R2 до и после преобразования

In [ ]:
r2 = cross_val_score(model, features, target, scoring='r2', cv=3).mean()
r2_enconded = cross_val_score(model, features_encoded, target, scoring='r2', cv=3).mean()

print('Качество линейной регрессии (R2) до преобразования:', round(r2, 10))
print('Качество линейной регрессии (R2) после преобразования:', round(r2_enconded, 10))
print('Разница в качестве до и после преобразования:', round(r2, 10) - round(r2_enconded, 10))

Качество линейной регрессии (R2) до преобразования: 0.4296932703
Качество линейной регрессии (R2) после преобразования: 0.4296932703
Разница в качестве до и после преобразования: 0.0


### Вывод:
 - Проверили на практике шифрование методом Хилла, который не нашёл практического применения в криптографии из-за слабой устойчивости ко взлому.